# Прогнозирование заказов такси

**Цель работы** — построить модель для предсказания количества заказов такси на следующий час, чтобы была возможность привлекать больше водителей в период пиковой нагрузки. По требованию от заказчика, предоставившего исторические данные о заказах такси в аэропортах за полгода, значение метрики RMSE не должно превышать 48.

**Ход работы**

Загрузим имеющиеся данные, при необходимости ресемплируем их.

Проанализируем получившиеся данные, после чего обучим на них несколько моделей, выбрав в итоге лучшую, подбирая в том числе их гиперпараметры.

Далее оценим качество модели на тестовых данных и сформулируем общий вывод.
 
В итоге работа будет состоять из следующих этапов:

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-обработка-данных" data-toc-modified-id="Загрузка-и-обработка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и обработка данных</a></span></li><li><span><a href="#Анализ-данных" data-toc-modified-id="Анализ-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ данных</a></span></li><li><span><a href="#Выбор-модели-и-гиперпараметров" data-toc-modified-id="Выбор-модели-и-гиперпараметров-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выбор модели и гиперпараметров</a></span><ul class="toc-item"><li><span><a href="#Обработка-данных-и-генерация-признаков" data-toc-modified-id="Обработка-данных-и-генерация-признаков-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Обработка данных и генерация признаков</a></span></li><li><span><a href="#Обучение-и-сравнение-моделей" data-toc-modified-id="Обучение-и-сравнение-моделей-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Обучение и сравнение моделей</a></span></li><li><span><a href="#Проверка-адекватности-модели-и-результатов-на-тестовой-выборке" data-toc-modified-id="Проверка-адекватности-модели-и-результатов-на-тестовой-выборке-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Проверка адекватности модели и результатов на тестовой выборке</a></span></li></ul></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Итоговый вывод</a></span></li></ul></div>

## Загрузка и обработка данных

In [2]:
import warnings

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from optuna.logging import disable_default_handler
from optuna.visualization import plot_param_importances
from optuna.integration import OptunaSearchCV
from optuna.distributions import \
    IntDistribution, \
    FloatDistribution, \
    CategoricalDistribution

SEED = 3
warnings.simplefilter('ignore', category=UserWarning)
disable_default_handler()

Предварительно ознакомимся с данными. При загрузке сразу сделаем даты индексом.

In [3]:
orders = pd.read_csv('/datasets/taxi.csv', index_col=[0], parse_dates=[0])

display(
    orders.sample(10, random_state=SEED),
    orders.info()
    )
print()
print(
    'Данные расположены в хронологическом порядке:',
    orders.index.is_monotonic
    )

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB


,num_orders
datetime,
2018-05-27 14:20:00,19
2018-03-01 00:50:00,21
2018-04-06 09:00:00,15
2018-03-07 02:30:00,2
2018-04-19 05:10:00,6
2018-06-30 11:50:00,15
2018-05-28 03:30:00,30
2018-03-19 13:20:00,6
2018-07-08 22:00:00,16


None


Данные расположены в хронологическом порядке: True


Данные предоставлены с интервалом в 10 мин, предсказания же нужны на следующий час. Проведём соответствующее ресемплирование.

In [4]:
orders = orders.resample('1H').sum()
display(
    orders.sample(10, random_state=SEED),
    orders.info()
    )

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4416 entries, 2018-03-01 00:00:00 to 2018-08-31 23:00:00
Freq: H
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  4416 non-null   int64
dtypes: int64(1)
memory usage: 69.0 KB


,num_orders
datetime,
2018-07-07 23:00:00,119
2018-04-16 22:00:00,73
2018-06-09 10:00:00,65
2018-03-19 09:00:00,73
2018-06-17 11:00:00,54
2018-08-01 12:00:00,72
2018-04-13 19:00:00,80
2018-03-09 21:00:00,70
2018-06-08 02:00:00,142


None

## Анализ данных

Взглянем на распределение данных, отобразим их скользящее среднее и тренды с сезонностью.

In [5]:
# средние показатели в час по дням
px.line(orders.resample('1D').mean()).update_layout(
    showlegend=False,
    xaxis_title='Дни',
    yaxis_title='Заказы в час',
    title=dict(
        text='Среднее число заказов такси в час по дням',
        x=.5
        )
    )

В течение полугода среднее число заказов такси в час достаточно плавно растет. Рассмотрим скользящее среднее без агрегации по дням.

In [6]:
px.line(orders.rolling(72).mean()).update_layout(
    showlegend=False,
    xaxis_title='Часы',
    yaxis_title='Заказы в час',
    title=dict(
        text='Скользящее среднее число заказов такси в час',
        x=.5
        )
    )

При достаточном сглаживании видно, что и в рамках отдельных месяцев, за исключением первого — марта, число заказов в час также растёт. Как вариант, это может быть связано с успешностью развития бизнеса компании или свидетельствовать о каких-либо сезонных закономерностях, но сделать выводы о последних в данный момент невозможно, так как имеются данные только за полгода.

Осталось взглянуть на тренды с сезонностью.

In [7]:
decomposed = seasonal_decompose(orders.resample('1D').mean())

fig = make_subplots(
    2,
    1,
    shared_xaxes=True,
    vertical_spacing=.1,
    subplot_titles=[
        'тренды',
        'сезонность'
        ]
    )
fig.add_trace(
    go.Scatter(
        y=decomposed.trend,
        x=decomposed.trend.index,
        mode='lines'
        ),
    1,
    1
    )
fig.add_trace(
    go.Scatter(
        y=decomposed.seasonal,
        x=decomposed.seasonal.index,
        mode='lines'
        ),
    2,
    1
    )
fig.update_layout(
    showlegend=False,
    height=750,
    title=dict(
        text='Тренды и сезонность заказов такси',
        x=.5
        )
    )
fig.show()

По графику трендов видно, что среднее число заказов в час чаще всего растёт в течение месяца, но к его концу несколько падает обратно, после чего растёт дальше до ещё более высоких значений в следующем месяце. Данная ситуация может быть связана, например, с тем, что в конце месяца у клиентов такси остаётся меньше свободных денежных средств и они начинают экономить.

Сезонность рассмотрим более детально.

In [8]:
px.line(
    y=decomposed.seasonal['2018-03-05':'2018-03-19'],
    x=decomposed.seasonal['2018-03-05':'2018-03-19'].index,
    ).update_layout(
        xaxis_title='Дни',
        yaxis_title='Заказы',
        title=dict(
            text='Сезонность заказов такси за две недели',
            x=.5
            )
        )

Видно, что пики числа заказов приходятся на понедельник и пятницу, возможно, частично это связано с рабочими/учебными графиками клиентов. В выходные трафик падает, минимум наблюдается во вторник с последующим увеличением заказов до пятницы включительно.

## Выбор модели и гиперпараметров

### Обработка данных и генерация признаков

Сооздадим признаки для дальнейшего обучения наших моделей, после чего разделим данные на обучающую и тестовую выборки.

Сгененрируем календарные признаки, предыдущие значения временного ряда для исторических показателей изменения функции, а также скользящее среднее для фиксации трендов.

In [9]:
def make_features(data, max_lag, rolling_mean_size):
    data['day'] = data.index.day
    data['day_of_week'] = data.index.dayofweek
    data['hour'] = data.index.hour
    
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    data['rolling_mean'] = data['num_orders'].rolling(
        rolling_mean_size,
        closed='left'
        ).mean()

make_features(orders, 24, 24)

Проверим получившиеся изменения наших данных.

In [10]:
display(
    orders.sample(10, random_state=SEED),
    orders.info()
    )

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4416 entries, 2018-03-01 00:00:00 to 2018-08-31 23:00:00
Freq: H
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   num_orders    4416 non-null   int64  
 1   day           4416 non-null   int64  
 2   day_of_week   4416 non-null   int64  
 3   hour          4416 non-null   int64  
 4   lag_1         4415 non-null   float64
 5   lag_2         4414 non-null   float64
 6   lag_3         4413 non-null   float64
 7   lag_4         4412 non-null   float64
 8   lag_5         4411 non-null   float64
 9   lag_6         4410 non-null   float64
 10  lag_7         4409 non-null   float64
 11  lag_8         4408 non-null   float64
 12  lag_9         4407 non-null   float64
 13  lag_10        4406 non-null   float64
 14  lag_11        4405 non-null   float64
 15  lag_12        4404 non-null   float64
 16  lag_13        4403 non-null   float64
 17  lag_14        4402 non-null

,num_orders,day,day_of_week,hour,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,...,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,rolling_mean
datetime,,,,,,,,,,,,,,,,,,,,,
2018-07-07 23:00:00,119,7,5,23,91.0,72.0,76.0,82.0,52.0,60.0,...,27.0,24.0,48.0,147.0,114.0,154.0,127.0,190.0,107.0,92.041667
2018-04-16 22:00:00,73,16,0,22,47.0,75.0,64.0,63.0,59.0,104.0,...,13.0,31.0,37.0,77.0,79.0,109.0,165.0,119.0,104.0,74.291667
2018-06-09 10:00:00,65,9,5,10,89.0,103.0,32.0,20.0,64.0,92.0,...,76.0,82.0,121.0,80.0,85.0,78.0,72.0,117.0,124.0,89.000000
2018-03-19 09:00:00,73,19,0,9,40.0,12.0,10.0,10.0,46.0,65.0,...,40.0,52.0,72.0,55.0,69.0,32.0,16.0,32.0,58.0,54.791667
2018-06-17 11:00:00,54,17,6,11,79.0,62.0,73.0,48.0,18.0,67.0,...,70.0,83.0,73.0,143.0,93.0,92.0,66.0,32.0,79.0,78.708333
2018-08-01 12:00:00,72,1,2,12,90.0,75.0,131.0,66.0,43.0,88.0,...,98.0,85.0,42.0,112.0,100.0,99.0,75.0,58.0,85.0,96.583333
2018-04-13 19:00:00,80,13,4,19,73.0,71.0,95.0,88.0,57.0,47.0,...,73.0,39.0,67.0,113.0,88.0,83.0,68.0,55.0,48.0,63.458333
2018-03-09 21:00:00,70,9,4,21,60.0,48.0,39.0,66.0,59.0,44.0,...,1.0,30.0,31.0,29.0,66.0,84.0,79.0,88.0,86.0,47.000000
2018-06-08 02:00:00,142,8,4,2,69.0,159.0,107.0,94.0,86.0,61.0,...,87.0,104.0,78.0,34.0,15.0,34.0,83.0,102.0,101.0,77.166667


None

Разделим данные на выборки, удалив небольшое количество записей с пропусками, образовавшихся в результате вычисления предыдущих значений временного ряда.

In [11]:
orders = orders.dropna()
features = orders.drop('num_orders', axis=1)
target = orders.num_orders

train_features, test_features, train_target, test_target = train_test_split(
    features,
    target,
    test_size=.1,
    random_state=SEED,
    shuffle=False
    )

Проверим корректность разделения, сравнив размеры выборок и их самые ранние и поздние даты.

In [12]:
print(
    'Размеры тренировочной и тестовой выборок с признаками: ',
    train_features.shape,
    test_features.shape,
    '\n',
    'Размеры тренировочной и тестовой выборок с целевым признаком: ',
    train_target.shape,
    test_target.shape
    )
print()
print(
    'Наиболее ранняя и поздняя даты тренировочной выборки: ',
    train_features.index.min(),
    train_features.index.max(),
    '\n',
    'Наиболее ранняя и поздняя даты тестовой выборки: ',
    test_features.index.min(),
    test_features.index.max()
    )

Размеры тренировочной и тестовой выборок с признаками:  (3952, 28) (440, 28) 
 Размеры тренировочной и тестовой выборок с целевым признаком:  (3952,) (440,)

Наиболее ранняя и поздняя даты тренировочной выборки:  2018-03-02 00:00:00 2018-08-13 15:00:00 
 Наиболее ранняя и поздняя даты тестовой выборки:  2018-08-13 16:00:00 2018-08-31 23:00:00


### Обучение и сравнение моделей

Сравним алгоритмы случайного леса и градиентного бустинга, подбирая для них наилучшие гиперпараметры, также используем линейную регрессию в качестве бейзлайна. 

Сначала тестово рассмотрим набор параметров на ограниченных диапазонах, после чего, оценив важность этих параметров для прогнозирования, окончательно подберём уже только наиболее важные из них на увеличенных диапазонах и количестве попыток. Это позволит как сократить итоговое время обучения, и так и добиться большего качества моделей.

In [13]:
metrics = pd.DataFrame(
    index=['RMSE'],
    columns=['LR', 'RF', 'GB']
    )
tss = TimeSeriesSplit()

In [14]:
%%time
lr_params = {}
lr_model = LinearRegression(n_jobs=-1)
best_lr_model = OptunaSearchCV(
    lr_model,
    lr_params,
    scoring='neg_root_mean_squared_error',
    cv=tss,
    n_trials=100,
    n_jobs=-1
    )
best_lr_model.fit(train_features, train_target)
metrics.loc['RMSE', 'LR'] = -best_lr_model.best_score_

<timed exec>:3: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.



CPU times: total: 1.38 s
Wall time: 2.34 s


In [15]:
%%time
rf_params = {
    'n_estimators': IntDistribution(100, 1000, step=100),
    'max_depth': IntDistribution(6, 30, step=3),
    'min_samples_leaf': IntDistribution(1, 28, step=3),
    'max_features': CategoricalDistribution(['sqrt', 'log2'])
    }
rf_model = RandomForestRegressor(random_state=SEED, n_jobs=-1)
best_rf_model = OptunaSearchCV(
    rf_model,
    rf_params,
    scoring='neg_root_mean_squared_error',
    cv=tss,
    n_trials=50,
    n_jobs=-1,
    random_state=SEED
    )
best_rf_model.fit(train_features, train_target)

# просмотр важности гиперпараметров
plot_param_importances(best_rf_model.study_).update_layout(
    xaxis_title='Важность для качества',
    yaxis_title='Гиперпараметры',
    title=dict(
        text='Важность гиперпараметров случайного леса',
        x=.5
        )
    ).show()
plot_param_importances(
    best_rf_model.study_,
    target=lambda x: x.duration.total_seconds()
).update_layout(
    xaxis_title='Важность для времени обучения',
    yaxis_title='Гиперпараметры',
    title=dict(
        text='Важность гиперпараметров случайного леса',
        x=.5
        )
    ).show()
print(
    'Лучшие параметры для случайного леса:',
    best_rf_model.best_params_
    )

<timed exec>:8: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.



Лучшие параметры для случайного леса: {'n_estimators': 800, 'max_depth': 30, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
CPU times: total: 25.1 s
Wall time: 3min 7s


In [16]:
%%time
# здесь и далее оставляем параметры, учитывая их влияние на предсказания, а также на длительность подбора
rf_params = {
    'max_features': CategoricalDistribution(['sqrt', 'log2']),
    'max_depth': IntDistribution(15, 40),
    'min_samples_leaf': IntDistribution(1, 10)
    }
rf_model = RandomForestRegressor(
    random_state=SEED,
    n_jobs=-1,
    n_estimators=800
    )
best_rf_model = OptunaSearchCV(
    rf_model,
    rf_params,
    scoring='neg_root_mean_squared_error',
    cv=tss,
    n_trials=100,
    n_jobs=-1,
    random_state=SEED
    )
best_rf_model.fit(train_features, train_target)

metrics.loc['RMSE', 'RF'] = -best_rf_model.best_score_
print(
    'Лучшие параметры для случайного леса:',
    best_rf_model.best_params_
    )

<timed exec>:12: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.



Лучшие параметры для случайного леса: {'max_features': 'sqrt', 'max_depth': 30, 'min_samples_leaf': 1}
CPU times: total: 51.5 s
Wall time: 7min 56s


In [17]:
%%time
categorical = [
    'day',
    'day_of_week',
    'hour'
    ]
gb_params = {
    'boosting_type': CategoricalDistribution(['gbdt', 'dart', 'goss']),
    'n_estimators': IntDistribution(100, 1000, step=50),
    'max_depth': IntDistribution(3, 30, step=3),
    'learning_rate': FloatDistribution(.01, .05),
    'reg_lambda': FloatDistribution(1, 5)
    }
gb_model = LGBMRegressor(random_state=SEED, n_jobs=-1)
best_gb_model = OptunaSearchCV(
    gb_model,
    gb_params,
    scoring='neg_root_mean_squared_error',
    cv=tss,
    n_trials=50,
    n_jobs=-1,
    random_state=SEED
    )
best_gb_model.fit(
    train_features,
    train_target,
    **{'categorical_feature': categorical}
    )
# просмотр важности гиперпараметров
plot_param_importances(best_gb_model.study_).update_layout(
    xaxis_title='Важность для качества',
    yaxis_title='Гиперпараметры',
    title=dict(
        text='Важность гиперпараметров градиентного бустинга',
        x=.5
        )
    ).show()
plot_param_importances(
    best_gb_model.study_,
    target=lambda x: x.duration.total_seconds()
).update_layout(
    width=1000,
    height=500,
    xaxis_title='Важность для времени обучения',
    yaxis_title='Гиперпараметры',
    title=dict(
        text='Важность гиперпараметров градиентного бустинга',
        x=.5
        )
    ).show()
print(
    'Лучшие параметры для градиентного бустинга:',
    best_gb_model.best_params_
    )

<timed exec>:14: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.



Лучшие параметры для градиентного бустинга: {'boosting_type': 'goss', 'n_estimators': 1000, 'max_depth': 3, 'learning_rate': 0.017319997221241577, 'reg_lambda': 4.595396155772667}
CPU times: total: 18min 18s
Wall time: 1min 24s


In [18]:
%%time
gb_params = {
    'reg_lambda': FloatDistribution(2, 6),
    'boosting_type': CategoricalDistribution(['gbdt', 'dart', 'goss']),
    'n_estimators': IntDistribution(800, 1200, step=10)
    }
gb_model = LGBMRegressor(
    random_state=SEED,
    n_jobs=-1,
    max_depth=3,
    learning_rate=.17
)
best_gb_model = OptunaSearchCV(
    gb_model,
    gb_params,
    scoring='neg_root_mean_squared_error',
    cv=tss,
    n_trials=100,
    n_jobs=-1,
    random_state=SEED
    )
best_gb_model.fit(
    train_features,
    train_target,
    **{'categorical_feature': categorical}
    )
metrics.loc['RMSE', 'GB'] = -best_gb_model.best_score_
print(
    'Лучшие параметры для градиентного бустинга:',
    best_gb_model.best_params_
    )

<timed exec>:12: ExperimentalWarning:

OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.



Лучшие параметры для градиентного бустинга: {'reg_lambda': 5.415731109183321, 'boosting_type': 'dart', 'n_estimators': 1020}
CPU times: total: 45min 31s
Wall time: 3min 24s


Сравним итоговые результаты моделей на кросс-валидации.

In [19]:
metrics

,LR,RF,GB
RMSE,27.15158,25.063064,24.922492


На уровне погрешности лучше оказывается модель градиентного бустинга. Её  в дальнейшем и протестируем окончательно.

### Проверка адекватности модели и результатов на тестовой выборке

При валидации модели показали неплохой результат. Посмотрим, насколько он сохранится на полностью новых данных, соответствует ли итоговое качество требованию от заказчика, а также насколько оно соотносится с различными dummy-вариантами решения.

In [20]:
predictions = best_gb_model.predict(test_features)
print(
    'RMSE на тестовых данных:',
    mean_squared_error(
        test_target,
        predictions,
        squared=False
        )
    )

RMSE на тестовых данных: 39.83823745812146


In [41]:
test_fig = go.Figure()
test_fig.add_trace(go.Bar(
    x=test_target.index,
    y=test_target,
    name='Факт',
    marker_color='dodgerblue'
    ))
test_fig.add_trace(go.Bar(
    x=test_target.index,
    y=predictions,
    name='Прогноз',
    marker_color='orange'
    ))
test_fig.update_layout(
    barmode='overlay',
    legend_orientation='h',
    title=dict(
        text='Истинное и предсказанное число заказов в час',
        x=.5
        ),
    xaxis_title='Час',
    yaxis_title='Количество заказов',
    margin=dict(t=25)
    )
test_fig.show()

Модель достаточно сильно переобучилась, разница в метрике по сравнению с валидацией около 1.5 раз. Но тем не менее она отвечает условию, заданному в начале — RMSE должна быть не более 48. Сравним этот результат с dummy-решениями.

Также видно, что предсказания модели достаточно равномерны по сравнению с истинными значениями, выбросов практически нет. То есть возможно случайные или обусловленные сложными закономерностями, не распознанными моделью, резко повышенные значения спроса она предсказать вряд ли сможет.

In [42]:
print(
    'RMSE при предсказаниях константой-медианой:',
    mean_squared_error(
        test_target,
        np.repeat(train_target.median(), test_target.shape[0]),
        squared=False
        )
    )
print()

preds = test_target.shift()
preds.iloc[0] = train_target.iloc[-1]
print(
    'RMSE при предсказаниях предыдущим значением:',
    mean_squared_error(
        test_target,
        preds,
        squared=False
        )
    )

RMSE при предсказаниях константой-медианой: 87.21093811714634

RMSE при предсказаниях предыдущим значением: 58.881776776551476


## Итоговый вывод

Результат итоговой модели и оказался лучше порога, заданного заказчиком, и прошёл проверку на адекватность. Между выбранными алгоритмами разницы по качеству практически не оказалось, между "лесными" моделями она и вовсе на уровне погрешности.

Но значение метрики не сильно ниже требуемого и в то же время значительно выше показателя на валидации, что свидетельствует о достаточном переобучении. Для его уменьшения, а также для дальнейшего улучшения метрики при необходимости можно попробовать другие значения гиперпараметров моделей, возможо, протестировать нейронные сети, увеличить количество фичей, как минимум с предыдущими значениями целевого признака, а также использовать данные за больший промежуток времени, нежели полгода.